In [1]:
from joblib import Memory

memory = Memory(".cache", verbose=0)

In [2]:
from transformers import BertTokenizer, BertModel
import torch
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

@memory.cache
def get_tokenizer_model():
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    bert_model = BertModel.from_pretrained('bert-base-cased')
    sbert_model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

    return bert_tokenizer, bert_model, sbert_model

bert_tokenizer, bert_model, sbert_model = get_tokenizer_model()

In [65]:
from pathlib import Path
import itertools
from natsort import natsorted

@memory.cache
def read_problem_files(problem_folder, n=None):
    """
    reads ground truth files into dict
    :param truth_folder: path to folder holding ground truth files
    :return: dict of ground truth files with problem-id as key and file content as value
    """
    problems = []
    files = itertools.islice(natsorted(Path(problem_folder).glob('problem-*.txt')), n)
    for problem_file in files:
        # number = problem_file.name[len("problem-") : -len(".txt")]
        with open(problem_file, 'r', encoding="utf8") as fh:
            problems.append(fh.readlines())
    return problems

In [62]:
from evaluation.evaluator import read_ground_truth_files

@memory.cache
def cached_read_ground_truth(x):
    return read_ground_truth_files(x)

ground_truth = cached_read_ground_truth("pan21/train")

In [117]:
def get_embeddings(paragraph):
    # Tokenize the sentence
    tokens = bert_tokenizer.tokenize(paragraph)
    token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)

    # Convert token IDs to tensor
    token_tensor = torch.tensor([token_ids])
    segment_ids = [0] * len(token_ids)
    segment_tensor = torch.tensor([segment_ids])

    # Get BERT model output
    with torch.no_grad():
        outputs = bert_model(token_tensor, segment_tensor)

    # Extract word embeddings from BERT model output
    return outputs[0]

In [66]:
problems = read_problem_files("pan21/train", n=5)
problems[0]

In [172]:
import torch

max_input_length = max([max([len(x) for x in read_problem_files("pan21/train", n=None)]), max([len(x) for x in read_problem_files("pan21/validation", n=None)])])

def pad_paragraph(paragraph_embedding, desired_length):
    d1, d2, d3 = paragraph_embedding.shape

    target = torch.zeros(d1, desired_length, d3)
    target[:, :d2, :] = paragraph_embedding

    return target


def get_paragraph_pairs(problem_text):
    # print(problem_text)
    paragraph_embeddings = [pad_paragraph(get_embeddings(para), desired_length) for para in problem_text]
    # print(f"{[x.shape for x in paragraph_embeddings]}")
    pairs = itertools.combinations(paragraph_embeddings, 2)
    return [torch.flatten(torch.stack(pair, dim=2), start_dim=1, end_dim=2) for pair in pairs]


In [173]:
from tqdm.notebook import tqdm

@memory.cache
def get_problem_embeddings(problems):
    return [get_paragraph_pairs(problem_text) for problem_text in tqdm(problems)]

In [174]:
problems_embed = get_problem_embeddings(problems)
for i in range(5):
    print(problems_embed[i][0][0].shape)
problems_embed[0][0].shape

  0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: stack expects each tensor to be equal size, but got [1, 121, 768] at entry 0 and [1, 87, 768] at entry 1

In [149]:
def get_simple_ground_truth(ground_truth, problem_numbers):
    simple_ground_truth = []
    for num in problem_numbers:
        task_3_ground_truth = ground_truth[f"problem-{num}"]["paragraph-authors"]
        simple_ground_truth.append(task_3_ground_truth)
    return simple_ground_truth


simple_ground_truth = get_simple_ground_truth(ground_truth, range(1, len(problems_embed)))

In [150]:
simple_ground_truth

[[1, 2, 2, 2, 2, 3, 2, 2],
 [1, 2, 2, 2, 1, 2],
 [1, 2, 2, 3, 3, 2],
 [1, 1, 1, 1, 1]]

In [151]:
def get_task_3_ground_truth(simple_ground_truth):
    task_gt = []
    for problem in simple_ground_truth:
        problem_gt = []
        for author1, author2 in itertools.combinations(problem, 2):
            problem_gt.append(int(author1 != author2))
        task_gt.append(problem_gt)
    return task_gt

task_3_ground_truth = get_task_3_ground_truth(simple_ground_truth)

In [152]:
# TODO: Invert the function get_task_3_ground_truth. Our model will output a bunch of binary labels which need to be converted to the task 3 ground truth format
# Ground truth format (gtf): [1, 2, 2, 2, 2, 3, 2, 2]
# Binary labels for comparisons (bl): [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0]
# Each binary label is the result of comparing two paragraphs. 1 means there was an author change, 0 means there was no author change
# For example, bl[0], is the result of comparing gtf[0]=1 and gtf[1]=2. 1 != 2, therefore bl[0] = 1. bl[1]=1 is the result of gtf[0] == gtf[2] (1 == 2)
def get_simple_ground_truth_from_task_3(task_3_ground_truth):
    pass

simple_ground_truth == get_simple_ground_truth_from_task_3(task_3_ground_truth)

False

In [153]:
for s, t in zip(simple_ground_truth, task_3_ground_truth):
    print(f"{s}")
    print(f"{t}")

[1, 2, 2, 2, 2, 3, 2, 2]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0]
[1, 2, 2, 2, 1, 2]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
[1, 2, 2, 3, 3, 2]
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# Add code to write out the embeddings (X) and ground truths (y) so we can train without having to rerun the preprocessing step

In [161]:
import numpy as np

x_train = get_problem_embeddings(read_problem_files("pan21/train", n=50))
y_train = np.array(get_task_3_ground_truth(get_simple_ground_truth(cached_read_ground_truth("pan21/train"), range(1, len(x_train)+1))))
x_val = np.array(get_problem_embeddings(read_problem_files("pan21/validation", n=15)))
y_val = np.array(get_task_3_ground_truth(get_simple_ground_truth(cached_read_ground_truth("pan21/validation"), range(1, len(x_val)+1))))
x_test, y_test = x_val, y_val

max_input_length = max([max([len(x) for x in x_train]), max([len(x) for x in x_val])])

KeyboardInterrupt: 

In [155]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed
 
# Code implementation of the RNN for sequence labeling
def create_rnn_model(vocab_size, num_labels, embedding_dim, lstm_units):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(LSTM(lstm_units, return_sequences=True))
    model.add(TimeDistributed(Dense(num_labels, activation='softmax')))
    return model
 
# Example usage of the RNN model
vocab_size = 30000  # Replace with the actual size of the vocabulary
num_labels = 2  # Replace with the actual number of entity labels
embedding_dim = 768
lstm_units = 64
max_sequence_length = 35
 
model = create_rnn_model(vocab_size, num_labels, embedding_dim, lstm_units)
model.build((1, max_input_length))
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)        │ (1, 120, 768)          │    23,040,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (1, 120, 64)           │       213,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_14             │ (1, 120, 2)            │           130 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,253,378 (88.70 MB)

 Trainable params: 23,253,378 (88.70 MB)

 Non-trainable params: 0 (0.00 B)

In [156]:
from tensorflow.keras import optimizers, losses, metrics


model.compile(
    optimizer=optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[metrics.SparseCategoricalAccuracy()],
)


In [157]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

ValueError: Unrecognized data type: x=[[(tensor([[[-0.1567, -0.0226,  0.1406,  ..., -0.5221, -0.2277,  0.1685],
         [-0.1196,  0.0534,  0.3339,  ..., -0.2394, -0.0062,  0.3519],
         [-0.2149, -0.1587,  0.1938,  ..., -0.5090, -0.3069,  0.3809],
         ...,
         [-0.2401,  0.0257,  0.1918,  ..., -0.3628, -0.4398,  0.1910],
         [-0.1210, -0.0047,  0.1605,  ..., -0.2694, -0.6690,  0.0306],
         [-0.2834, -0.1104,  0.1601,  ..., -0.0979, -0.3949,  0.0611]]]), tensor([[[-0.0015,  0.1751, -0.0197,  ..., -0.3428, -0.1279, -0.0088],
         [ 0.0199,  0.0478, -0.0322,  ..., -0.1488, -0.2279, -0.0399],
         [-0.2033,  0.1145, -0.0892,  ..., -0.2715, -0.4363, -0.0912],
         ...,
         [ 0.2032,  0.1766,  0.0505,  ..., -0.2270, -0.3289,  0.0836],
         [-0.2388,  0.4306, -0.1116,  ..., -0.2458, -0.5069,  0.0334],
         [-0.0923,  0.1058, -0.0332,  ...,  0.0263, -0.3393, -0.0340]]])), (tensor([[[-0.1567, -0.0226,  0.1406,  ..., -0.5221, -0.2277,  0.1685],


In [140]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)


Evaluate on test data


ValueError: Unrecognized data type: x=[[(tensor([[[ 0.3419,  0.0934,  0.1832,  ..., -0.4819,  0.0700,  0.2659],
         [ 0.8806, -0.3243,  0.2881,  ..., -0.3994, -0.1309,  0.7042],
         [ 0.3406, -0.0658,  0.3373,  ..., -0.4040,  0.1481,  0.4763],
         ...,
         [ 0.4780, -0.2780, -0.2039,  ..., -0.3263, -0.2358,  0.2634],
         [ 0.7074, -0.1787,  0.0871,  ..., -0.5364, -0.1564,  0.3702],
         [ 0.4026,  0.0646,  0.2643,  ..., -0.0019,  0.0086,  0.2726]]]), tensor([[[ 0.2685,  0.0025,  0.3448,  ..., -0.8208,  0.1284,  0.0266],
         [ 0.3464, -0.0962,  0.5198,  ..., -1.0013,  0.1822,  0.0776],
         [ 0.2711, -0.0201,  0.4055,  ..., -0.9232,  0.0661,  0.2379],
         ...,
         [ 0.5292,  0.0113,  0.5495,  ..., -1.1265,  0.0595,  0.2168],
         [ 0.0451, -0.0028,  0.3217,  ..., -0.7201, -0.3509,  0.0911],
         [ 0.2849, -0.0897,  0.4328,  ..., -0.7129, -0.0783,  0.0607]]])), (tensor([[[ 0.3419,  0.0934,  0.1832,  ..., -0.4819,  0.0700,  0.2659],
